## Processing Ocean Color Imagery - Process Micasense Images

Pull in all images and process with micasense tools
    normalize and align and whatnot
    put these into functions and clean up
    also visualize along the way and look at distributions to think about whether or not they're legitimate
    write out
    
Pull in all images as modified and metadata with specific functions
    Visualize for issues and shadows and bright spots and locations
    correct for sky radiance
    visualize the distributions of both sky (clouds) and sea
    look at the variability of spectra both for specific pixels and across nearby images
    filter for orientation and location and issues

take the water leaving spectra and convert to chla as well as other future products and just intro general analysis of spectra
    produce a csv or .shp or images of ocean color products 
    

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from ipywidgets import FloatProgress, Layout
from IPython.display import display
from micasense import imageset as imageset
from micasense import capture as capture
import multiprocessing
import glob, shutil, os

import cv2
import numpy as np
import matplotlib.pyplot as plt


import micasense.imageutils as imageutils
import micasense.plotutils as plotutils

import exiftool
import datetime

import subprocess
import math

from utils import *
from georeg import *

Inspect the Exposure, gain, DLS

In [ ]:
imgset = imageset.ImageSet.from_directory(alt_dir)

In [ ]:
dls_ed = []
exposures = []
gains = []
for cap in imgset.captures:
    if len(cap.dls_irradiance()) == 6:
        dls_ed.append(np.array(cap.dls_irradiance()))
        exp = []
        gain_cap = []
        for img in cap.images: 
            exp.append(img.exposure_time)
            gain_cap.append(img.gain)
        exposures.append(exp)
        gains.append(gain_cap)
        
exposures = np.array(exposures)
gains = np.array(gains)
dls_ed = np.array(dls_ed)

In [ ]:
band_names = ['blue', 'green', 'red', 'nir', 'red edge']

fig, ax = plt.subplots(5,1, figsize=(12,18))

for i in range(5):
    ax[i].plot(exposures[100:280,i], color='green')
    ax[i].set_ylim(0,.002)
    ax2 = ax[i].twinx()
    ax2.plot(gains[100:280,i], color='blue')
    ax2.set_ylim(0,8.2)
    ax3 = ax[i].twinx()
    ax3.plot(dls_ed[100:280,i], color='red')
#     ax3.set_ylim(.22,.5)
    ax3.spines['right'].set_position(('outward', 60))
    
    ax[i].set_ylabel('exposure')
    ax2.set_ylabel('gain')
    ax3.set_ylabel('DLS Reading')
    
    ax[i].yaxis.label.set_color('green')
    ax2.yaxis.label.set_color('blue')
    ax3.yaxis.label.set_color('red')
    
    ax[i].set_title(band_names[i])
    fig.tight_layout()
    fig.savefig('gain_exposure_dls.png',dpi=300)
    fig.show()
#     

### Run the process

In [5]:
! ls data/210827_A1_frontcross1/

0000SET  panel	RedEdge  sea  SYNLog-13-39-33_27-08-2021.csv


### This is the typical full workflow

In [9]:
glob.glob(os.path.join('data', '210827_A1_frontcross1', 'panel', 'IMG_0006*.tif'))

['data/210827_A1_frontcross1/panel/IMG_0006_4.tif',
 'data/210827_A1_frontcross1/panel/IMG_0006_2.tif',
 'data/210827_A1_frontcross1/panel/IMG_0006_3.tif',
 'data/210827_A1_frontcross1/panel/IMG_0006_1.tif',
 'data/210827_A1_frontcross1/panel/IMG_0006_6.tif',
 'data/210827_A1_frontcross1/panel/IMG_0006_5.tif']

#### First process the Altum images

From https://support.micasense.com/hc/en-us/articles/360025336894-Using-Panels-and-or-DLS-in-Post-Processing

In the case of a clear, sunny day, we recommend using only the calibration panels. Light sensor data will not be as effective and so should generally not be used in this case.

Light sensor data is most effective in overcast, completely cloudy conditions. In this case, we would recommend using both light sensor and panel data for calibration.

In [11]:
# to use the DLS and not the panel set this to None
# panelNames = None
panelNames = glob.glob(os.path.join('data', '210827_A1_frontcross1', 'panel', 'IMG_0006*.tif'))

surface_dir = os.path.join('data', '210827_A1_frontcross1', 'sea')
warp_img_dir = os.path.join('data', '210827_A1_frontcross1', 'sea', 'align_img')
surface_out = process_micasense_subset(surface_dir, panelNames, warp_img_dir, overwrite=False)

Panel irradiance calculated.
used warp dir data/210827_A1_frontcross1/sea/align_img
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 0
Finished aligning band 4
Finished aligning band 2
Finished aligning band 1
Finished aligning band 3
Saving time: 0:34:08.950945
Alignment+Saving rate: 0.25 images per second
Finished saving images.
exiftool -csv="data/210827_A1_frontcross1/sea/stacks/log.csv" -overwrite_original data/210827_A1_frontcross1/sea/stacks
Finished saving image metadata.


In [12]:
! exiftool -csv="data/210827_A1_frontcross1/sea/stacks/log.csv" -overwrite_original data/210827_A1_frontcross1/sea/stacks

    1 directories scanned
  507 image files updated


#### Now process the RedEdge Images

Need to ensure these are the correct images via time. They do seem correct.

Also note that the first couple sea images were bad because of spray and glint and shadow.

In [15]:
! ls data/202108

panel  sea  sky  tube


In [13]:
# to use the DLS and not the panel set this to None
#panelNames = None
panelNames = glob.glob(os.path.join('data', '210827_A1_frontcross1', 'RedEdge', 'panel', 'IMG_0000*.tif'))

blocked_dir = os.path.join('data', '210827_A1_frontcross1', 'RedEdge', 'tube')
sky_dir = os.path.join('data', '210827_A1_frontcross1', 'RedEdge', 'sky')
surface_dir = os.path.join('data', '210827_A1_frontcross1', 'RedEdge', 'sea')

warp_img_dir = os.path.join('data', '210827_A1_frontcross1', 'rededge', 'tube')

sunlight_blocked_out = process_micasense_subset(blocked_dir, panelNames, warp_img_dir, overwrite=False)
# sky_out = process_micasense_subset(sky_dir, panelNames, warp_img_dir, overwrite=False)
# surface_out = process_micasense_subset(surface_dir, panelNames, warp_img_dir, overwrite=False)

Panel irradiance calculated.
used warp dir data/210827_A1_frontcross1/RedEdge/tube
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 0
Finished aligning band 3
Finished aligning band 2
Finished aligning band 4
Finished aligning band 1
Saving time: 0:00:14.567565
Alignment+Saving rate: 0.69 images per second
Finished saving images.
exiftool -csv="data/210827_A1_frontcross1/RedEdge/tube/stacks/log.csv" -overwrite_original data/210827_A1_frontcross1/RedEdge/tube/stacks
Finished saving image metadata.
Panel irradiance calculated.


ValueError: Provided file path does not have a band name: data/210827_A1_frontcross1/RedEdge/tube/stacks/J3EylIoPlnSU2lQMkE0D.tif

In [15]:
warp_img_dir = os.path.join('data', '210827_A1_frontcross1', 'RedEdge', 'sky')
sky_out = process_micasense_subset(sky_dir, panelNames, warp_img_dir, overwrite=False)

warp_img_dir = os.path.join('data', '210827_A1_frontcross1', 'RedEdge', 'sea')
surface_out = process_micasense_subset(surface_dir, panelNames, warp_img_dir, overwrite=False)

Panel irradiance calculated.
used warp dir data/210827_A1_frontcross1/RedEdge/sky
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 0
Finished aligning band 1
Finished aligning band 4
Finished aligning band 3
Finished aligning band 2
Saving time: 0:00:05.766430
Alignment+Saving rate: 0.87 images per second
Finished saving images.
exiftool -csv="data/210827_A1_frontcross1/RedEdge/sky/stacks/log.csv" -overwrite_original data/210827_A1_frontcross1/RedEdge/sky/stacks
Finished saving image metadata.
Panel irradiance calculated.
used warp dir data/210827_A1_frontcross1/RedEdge/sea
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 0
Finished aligning band 1
Finished aligning band 2
Finished aligning band 4
Finished aligning band 3
Saving time: 0:00:03.828400
Alignment+Saving rate: 0.78 images per second
Finished saving images.
exiftool -csv="data/210827_A1_frontcross1/RedE

In [17]:
! exiftool -csv="data/210827_A1_frontcross1/RedEdge/tube/stacks/log.csv" -overwrite_original data/210827_A1_frontcross1/RedEdge/tube/stacks

    1 directories scanned
   10 image files updated


In [19]:
! exiftool -csv="data/210827_A1_frontcross1/RedEdge/sky/stacks/log.csv" -overwrite_original data/210827_A1_frontcross1/RedEdge/sky/stacks

    1 directories scanned
    5 image files updated


In [20]:
! exiftool -csv="data/210827_A1_frontcross1/RedEdge/sea/stacks/log.csv" -overwrite_original data/210827_A1_frontcross1/RedEdge/sea/stacks

    1 directories scanned
    3 image files updated
